In [92]:
import json
import pandas as pd
import numpy as np
from pathlib import Path

# === 파일 경로 (노트북 실행 폴더 기준) ===
# 같은 폴더에 데이터 파일 넣어두면 됨
PARKS_JSON   = Path("서울시 주요 공원현황.json")
POP_JSON     = Path("250810_행정동 단위 서울 생활인구(내국인).json")
DONG_XY_XLSX = Path("서울시_행정동_좌표.csv")

# === 분석 파라미터 ===
RADII_M = [500, 1000, 2000]     # 500m / 1km / 2km
HOURS   = [f"{h:02d}" for h in range(7, 23)]  # 07~23시
DATES_TO_INCLUDE = None         # 특정 날짜만 분석하고 싶으면 ["20240801", "20240815"] 이런 식으로

# === 공원 목록 (24개) ===
parks_target_tour = [
    "도산근린공원", "율현공원", "경의선숲길", "문화비축기지", "올림픽공원",
    "송파나루근린공원(석촌호수)", "인왕산도시자연공원(인왕산)", "낙산공원",
    "북한산국립공원(북한산)", "서울로7017", "남산공원", "용산가족공원",
    "효창근린공원(효창공원)",
]
parks_target_han = [
    "뚝섬한강공원","잠원한강공원","여의도한강공원","반포한강공원","망원한강공원",
    "잠실한강공원","양화한강공원","난지한강공원","이촌한강공원","광나루한강공원","강서한강공원"
]
parks_target_add = [
    "길동생태공원", "일자산허브천문공원", "북서울꿈의숲", "방화근린공원", "우장산근린공원(우장산/우장산근린공원)", "서울식물원", 
    "서울대공원", "관악산공원(관악산)", "어린이대공원", "아차산공원(아차산)", "고척근린공원", "푸른수목원", "금천체육공원", "금천폭포근린공원", 
    "수락산도시자연공원(수락산)", "불암산도시자연공원(불암산)", "서울창포원", "용두근린공원(용두공원)", "배봉산근린공원(배봉산)", 
    "국립서울현충원", "보라매공원", "독립공원(서대문독립공원)", "청계산도시자연공원(청계산)", "매헌시민의숲", "서울숲", "달맞이근린공원", 
    "청량근린공원(천장산)", "서서울호수공원", "파리근린공원(파리공원)", "선유도공원", "여의도공원(여의도도시근린공원)", 
    "진관근린공원(구파발폭포/이말산)", "사가정공원(용마산)", "중랑캠핑숲(중랑가족캠핑장)"
    
]
PARKS_TARGET = parks_target_tour + parks_target_han + parks_target_add


In [93]:
from math import radians, sin, cos, asin, sqrt
import numpy as np
import pandas as pd
from pathlib import Path

def norm_text(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip()
    return s.replace(" ", "")

def haversine_vec(lat1_arr, lon1_arr, lat2, lon2):
    """벡터화 하버사인 거리(m)"""
    R = 6371000.0
    lat1 = np.radians(lat1_arr); lon1 = np.radians(lon1_arr)
    lat2 = np.radians(lat2);     lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# 공원명 별칭(데이터셋 표기 차이를 흡수)
PARK_ALIASES = {
    "송파나루근린공원(석촌호수)": ["석촌호수","송파나루공원"],
    "인왕산도시자연공원(인왕산)": ["인왕산도시자연공원","인왕산"],
    "북한산국립공원(북한산)": ["북한산국립공원","북한산"],
    "서울로 7017": ["서울로7017"],
    "낙산공원": ["낙산공원", "낙산근린공원"],
    "효창근린공원(효창공원)": ["효창공원", "효창근린공원"],
    "훈련원근린공원(훈련원공원)": ["훈련원근린공원"],
    "북악산도시자연공원(북악산)": ["북악산도시자연공원"],
    '삼청근린공원(삼청공원)':["삼청근린공원"], 
    '오금근린공원(오금오름공원/오금공원)':["오금근린공원"], 
    '아시아근린공원(아시아공원)':["아시아근린공원"], 
    '장지근린공원(장지공원)':["장지근린공원"], 
    '도곡근린공원(도곡공원)':["도곡근린공원"], 
    '안산도시자연공원(안산)':["안산도시자연공원"], 
    '백련근린공원(백련산)':["백련근린공원"], 
    '독립공원(서대문독립공원)':["독립공원"], 
    '궁동근린공원(궁동공원)':["궁동근린공원"],
    '성북근린공원(성북공원)':['성북근린공원'], 
    '개운산근린공원(개운산)':["개운산근린공원"], 
    '청량근린공원(천장산)':["청량근린공원"], 
    '오동근린공원(오패산/오동공원)':["오동근린공원"], 
    '수락산도시자연공원(수락산)':["수락산도시자연공원"], 
    '불암산도시자연공원(불암산)':["불암산도시자연공원"], 
    '용두근린공원(용두공원)':["용두근린공원"], 
    '배봉산근린공원(배봉산)':["배봉산근린공원"], 
    '간데메근린공원(간데메공원)':["간데메근린공원"], 
    '답십리근린공원(답십리공원)':["답십리근린공원"], 
    '대현산배수지공원(응봉근린공원)':["대현산배수지공원"], 
    '성수근린공원(성수동구두테마공원)':["성수근린공원"], 
    '중랑캠핑숲(중랑가족캠핑장)':["중랑캠핑숲"], 
    '봉화산근린공원(봉화산)':["봉화산근린공원"], 
    '사가정공원(용마산)':["사가정공원"], 
    '아차산공원(아차산)':["아차산공원"], 
    '봉제산공원(봉제산근린공원/봉제산)':["봉제산공원"], 
    '개화근린공원(개화산)':["개화근린공원"], 
    '염창근린공원(염창산/증미산)':["염창근린공원"], 
    '우장산근린공원(우장산/우장산근린공원)':["우장산근린공원"], 
    '허준공원(허준근린공원)':["허준공원"], 
    '궁산근린공원(궁산)':["궁산근린공원"], 
    '꿩고개근린공원(치현산)':["꿩고개근린공원"], 
    '용왕산근린공원(용왕산)':["용왕산근린공원"], 
    '파리근린공원(파리공원)':["파리근린공원"], 
    '갈산근린공원(갈산공원/갈산)':["갈산근린공원"], 
    '개웅산근린공원(개웅산)':["개웅산근린공원"], 
    '중마루근린공원(중마루공원)':["중마루근린공원"], 
    '여의도공원(여의도도시근린공원)':["여의도공원"], 
    '영등포근린공원(영등포공원)':["영등포근린공원"], 
    '사육신공원(사육신역사공원)':["사육신공원"], 
    '삼일근린공원(삼일공원)':["삼일근린공원"], 
    '관악산공원(관악산)':["관악산공원"], 
    '문화예술공원(서초문화예술공원)':["문화예술공원"], 
    '청계산도시자연공원(청계산)':["청계산도시자연공원"], 
    '인능산도시자연공원(인릉산)':["인능산도시자연공원"], 
    '명일근린공원(명일공원)':["명일근린공원"], 
    '일자산근린공원(일자산)':["일자산근린공원"],
    '진관근린공원(구파발폭포/이말산)':["진관근린공원"],
}

def name_candidates(name: str):
    cs = [name, norm_text(name)]
    for k, alist in PARK_ALIASES.items():
        if name == k or name in alist:
            cs += alist + [norm_text(x) for x in alist]
    return list(dict.fromkeys(cs))  # 중복 제거


In [94]:
import json

# --- 공원 현황 로드 ---
with open(PARKS_JSON, "r", encoding="utf-8") as f:
    parks_json = json.load(f)

# 리스트/딕셔너리 모두 대응
if isinstance(parks_json, list):
    parks = pd.json_normalize(parks_json, max_level=2)
elif isinstance(parks_json, dict):
    list_keys = [k for k, v in parks_json.items() if isinstance(v, list)]
    key = list_keys[0] if list_keys else None
    parks = pd.json_normalize(parks_json[key] if key else parks_json, max_level=2)
else:
    raise ValueError("알 수 없는 공원 JSON 구조")

# 공원명/좌표 컬럼 자동 탐지 + 보정
name_col = next((c for c in parks.columns if "공원" in c or "name" in c.lower() or "p_park" in c.lower()), parks.columns[0])
lat_col  = next((c for c in parks.columns if "lat" in c.lower() or "위도" in c), None)
lng_col  = next((c for c in parks.columns if "lng" in c.lower() or "lon" in c.lower() or "경도" in c), None)
if "p_park" in parks.columns: name_col = "p_park"
if "latitude" in parks.columns: lat_col = "latitude"
if "longitude" in parks.columns: lng_col = "longitude"

parks["_name_raw"] = parks[name_col].astype(str)
parks["_name_norm"] = parks["_name_raw"].map(norm_text)
parks["_lat"] = pd.to_numeric(parks[lat_col], errors="coerce")
parks["_lng"] = pd.to_numeric(parks[lng_col], errors="coerce")
parks_clean = parks.dropna(subset=["_lat","_lng"]).copy()

# --- 타겟 24개 매칭 ---
target_rows, missing = [], []
for target in PARKS_TARGET:
    cands = name_candidates(target)
    sub = parks_clean[parks_clean["_name_raw"].isin(cands) | parks_clean["_name_norm"].isin([norm_text(x) for x in cands])]
    if len(sub) == 0:
        missing.append(target)
    else:
        target_rows.append(sub.iloc[0])

parks_24 = pd.DataFrame(target_rows)
print(f"매칭된 공원 수: {len(parks_24)} / {len(PARKS_TARGET)}")
if missing:
    print("⚠️ 매칭 실패 공원:", missing)

display(parks_24[["_name_raw","_lat","_lng"]].reset_index(drop=True))


매칭된 공원 수: 58 / 58


,_name_raw,_lat,_lng
0,도산근린공원,37.524675,127.035030
1,율현공원,37.472332,127.115594
2,경의선숲길,37.558887,126.925427
3,문화비축기지,37.571718,126.893246
4,올림픽공원,37.520934,127.122959
5,송파나루근린공원(석촌호수),37.506933,127.098300
6,인왕산도시자연공원,37.584900,126.957844
7,낙산근린공원,37.580477,127.007587
8,북한산국립공원,37.624549,127.000938
9,서울로7017,37.556714,126.969575


In [95]:
# --- 생활인구 로드 ---
import json, pandas as pd, numpy as np
from pathlib import Path

with open(POP_JSON, "r", encoding="utf-8") as f:
    pop_json = json.load(f)

# JSON → DataFrame (list/dict 모두 대응)
if isinstance(pop_json, list):
    pop = pd.json_normalize(pop_json, max_level=2)
elif isinstance(pop_json, dict):
    list_keys = [k for k, v in pop_json.items() if isinstance(v, list)]
    key = list_keys[0] if list_keys else None
    pop = pd.json_normalize(pop_json[key] if key else pop_json, max_level=2)
else:
    raise ValueError("알 수 없는 생활인구 JSON 구조")

# 인구 수치 합산('생활인구'가 있으면 우선 사용, 없으면 모든 수치형 합)
num_cols = [c for c in pop.columns if pd.api.types.is_numeric_dtype(pop[c])]
pop["pop"] = pop["생활인구"] if "생활인구" in pop.columns else (pop[num_cols].sum(axis=1) if num_cols else np.nan)

# 표준 컬럼명 가정(서울 생활인구 표준)
code_col = "adstrd_code_se" if "adstrd_code_se" in pop.columns else None
date_col = "stdr_de_id"     if "stdr_de_id" in pop.columns     else None
hour_col = "tmzon_pd_se"    if "tmzon_pd_se" in pop.columns    else None
if not all([code_col, date_col, hour_col]):
    raise ValueError("생활인구 JSON에 adstrd_code_se / stdr_de_id / tmzon_pd_se 컬럼이 필요합니다.")

# 날짜/시간 필터
if DATES_TO_INCLUDE:
    pop = pop[pop[date_col].astype(str).isin([str(d) for d in DATES_TO_INCLUDE])]
pop = pop[pop[hour_col].astype(str).isin(HOURS)].copy()

# ✅ 코드 8자리화 (중요!)
pop["code8"] = pop[code_col].astype(str).str[:8]

# --- 행정동 좌표 로드 (XLSX/CSV 자동 대응 + 인코딩 안전) ---
def read_dongxy(path: Path) -> pd.DataFrame:
    if path.suffix.lower() in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    elif path.suffix.lower() == ".csv":
        last_err = None
        for enc in ["utf-8-sig", "cp949", "euc-kr"]:
            try:
                return pd.read_csv(path, encoding=enc)
            except Exception as e:
                last_err = e
        raise last_err
    else:
        raise ValueError("행정동 좌표 파일은 .xlsx/.xls/.csv 중 하나여야 합니다.")

dongxy_raw = read_dongxy(DONG_XY_XLSX)

# 열 이름 정규화
rename_map = {}
cols_lower = {c: c.lower() for c in dongxy_raw.columns}
for c, lc in cols_lower.items():
    if lc in ["code","adm_cd","h_code","adstrd_code_se","법정동코드","행정동코드"]:
        rename_map[c] = "code"; break
for c, lc in cols_lower.items():
    if lc in ["lat","latitude","위도","y","center_lat","centroid_lat"]:
        rename_map[c] = "dong_lat"
for c, lc in cols_lower.items():
    if lc in ["lng","lon","longitude","경도","x","center_lng","centroid_lng"]:
        rename_map[c] = "dong_lng"
for c in dongxy_raw.columns:
    if c in ["행정동","법정동","동","adm_nm","dong","dong_name"]:
        rename_map[c] = "행정동"; break
for c in dongxy_raw.columns:
    if c in ["자치구","구","sgg_nm","gu","sgg_name"]:
        rename_map[c] = "자치구"; break

dongxy = dongxy_raw.rename(columns=rename_map).copy()

# 필수 컬럼 점검 + 정리
required_cols = ["code","dong_lat","dong_lng","행정동","자치구"]
missing = [c for c in required_cols if c not in dongxy.columns]
if missing:
    raise ValueError(f"행정동 좌표 파일에 필수 컬럼 누락: {missing}")

dongxy["code"] = dongxy["code"].astype(str)
dongxy["dong_lat"] = pd.to_numeric(dongxy["dong_lat"], errors="coerce")
dongxy["dong_lng"] = pd.to_numeric(dongxy["dong_lng"], errors="coerce")
dongxy = dongxy.dropna(subset=["dong_lat","dong_lng"]).copy()
dongxy["code8"] = dongxy["code"].str[:8]  # 10자리 → 8자리

# --- 조인 ---
popxy = pd.merge(
    pop[[date_col, hour_col, "pop", "code8"]],
    dongxy[["code8","행정동","자치구","dong_lat","dong_lng"]],
    on="code8", how="inner"
)

# 날짜/동 기준 시간대 합계(원하면 mean으로 변경 가능)
popxy_agg = (popxy
             .groupby([date_col,"code8","자치구","행정동","dong_lat","dong_lng"], as_index=False)["pop"]
             .sum())

print("집계 완료 → 날짜 수:", popxy_agg[date_col].nunique(), "/ 레코드:", len(popxy_agg))
display(popxy_agg.head())


집계 완료 → 날짜 수: 1 / 레코드: 418


,stdr_de_id,code8,자치구,행정동,dong_lat,dong_lng,pop
0,20250810,11110515,종로구,청운효자동,37.584009,126.970626,472313.4855
1,20250810,11110530,종로구,사직동,37.575408,126.965944,741114.9957
2,20250810,11110540,종로구,삼청동,37.590758,126.980996,180841.1124
3,20250810,11110550,종로구,부암동,37.594768,126.965574,326543.7806
4,20250810,11110560,종로구,평창동,37.613029,126.974485,584568.1402


In [96]:
def compute_popweighted_distance(parks_df, pop_agg_df, radius_m: int, date_col="stdr_de_id"):
    rows = []
    for date_val, grp in pop_agg_df.groupby(date_col):
        dong_lat = grp["dong_lat"].to_numpy()
        dong_lng = grp["dong_lng"].to_numpy()
        dong_pop = grp["pop"].to_numpy()

        for _, p in parks_df.iterrows():
            plat, plng, pname = float(p["_lat"]), float(p["_lng"]), str(p["_name_raw"])
            dists = haversine_vec(dong_lat, dong_lng, plat, plng)
            m = dists <= radius_m
            if m.any() and dong_pop[m].sum() > 0:
                wavg = float((dong_pop[m] * dists[m]).sum() / dong_pop[m].sum())
                total_pop = float(dong_pop[m].sum())
                near_cnt = int(m.sum())
            else:
                wavg, total_pop, near_cnt = np.nan, 0.0, 0

            rows.append({
                "날짜": str(date_val),
                "공원명": pname,
                "반경(m)": radius_m,
                "반경내_행정동수": near_cnt,
                "반경내_생활인구합": total_pop,
                "인구가중_평균거리(m)": wavg,
                "공원_lat": plat,
                "공원_lng": plng
            })
    return pd.DataFrame(rows)


In [97]:
# 출력 폴더(현재 폴더 하위) 생성
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

all_results = []
for r in RADII_M:
    res_r = compute_popweighted_distance(parks_24, popxy_agg, r)
    all_results.append(res_r)
    out_path = OUTPUT_DIR / f"250810_park_popweighted_distance_admin_dong_{r}m.csv"
    res_r.to_csv(out_path, index=False, encoding="utf-8-sig")
    print("저장:", out_path)

result_all = pd.concat(all_results, ignore_index=True)
out_all = OUTPUT_DIR / "250810_park_popweighted_distance_admin_dong_allradii.csv"
result_all.to_csv(out_all, index=False, encoding="utf-8-sig")
print("통합 저장:", out_all)

display(result_all.head(20))


저장: outputs/250810_park_popweighted_distance_admin_dong_500m.csv
저장: outputs/250810_park_popweighted_distance_admin_dong_1000m.csv
저장: outputs/250810_park_popweighted_distance_admin_dong_2000m.csv
통합 저장: outputs/250810_park_popweighted_distance_admin_dong_allradii.csv


,날짜,공원명,반경(m),반경내_행정동수,반경내_생활인구합,인구가중_평균거리(m),공원_lat,공원_lng
0,20250810,도산근린공원,500,0,0.0000,NaN,37.524675,127.035030
1,20250810,율현공원,500,0,0.0000,NaN,37.472332,127.115594
2,20250810,경의선숲길,500,0,0.0000,NaN,37.558887,126.925427
3,20250810,문화비축기지,500,0,0.0000,NaN,37.571718,126.893246
4,20250810,올림픽공원,500,0,0.0000,NaN,37.520934,127.122959
5,20250810,송파나루근린공원(석촌호수),500,0,0.0000,NaN,37.506933,127.098300
6,20250810,인왕산도시자연공원,500,0,0.0000,NaN,37.584900,126.957844
7,20250810,낙산근린공원,500,1,773699.5057,398.924572,37.580477,127.007587
8,20250810,북한산국립공원,500,0,0.0000,NaN,37.624549,127.000938
9,20250810,서울로7017,500,1,256318.9144,371.949523,37.556714,126.969575


In [98]:
summary = (result_all
           .groupby(["공원명","반경(m)"], as_index=False)["인구가중_평균거리(m)"]
           .mean(numeric_only=True)
           .sort_values(["반경(m)","인구가중_평균거리(m)"]))

display(summary.head(30))


,공원명,반경(m),인구가중_평균거리(m)
90,서울숲,500,96.793439
114,어린이대공원,500,103.389362
126,용두근린공원,500,174.702837
165,파리근린공원,500,190.151244
171,효창근린공원,500,282.272747
36,달맞이근린공원,500,317.850550
87,서울로7017,500,371.949523
42,독립공원,500,397.519601
27,낙산근린공원,500,398.924572
153,중랑캠핑숲,500,473.749720


In [99]:
print("▶ 공원 원본 행수:", len(parks))
print("▶ 좌표 있는 공원 행수:", len(parks_clean))
print("▶ 타겟 24 매칭:", len(parks_24), "/ 58")

if len(parks_24) < 24:
    missing_now = set(PARKS_TARGET) - set(parks_24["_name_raw"])
    print("⚠️ 매칭 실패 공원:", list(missing_now)[:10])

print("\n샘플 공원 5개")
display(parks_24[["_name_raw","_lat","_lng"]].head())

# 좌표 형식/범위 점검
print("\n좌표 NaN 개수:", parks_24["_lat"].isna().sum(), parks_24["_lng"].isna().sum())
print("위도 범위:", parks_24["_lat"].min(), "~", parks_24["_lat"].max())
print("경도 범위:", parks_24["_lng"].min(), "~", parks_24["_lng"].max())


▶ 공원 원본 행수: 151
▶ 좌표 있는 공원 행수: 151
▶ 타겟 24 매칭: 58 / 58

샘플 공원 5개


,_name_raw,_lat,_lng
45,도산근린공원,37.524675,127.035030
128,율현공원,37.472332,127.115594
131,경의선숲길,37.558887,126.925427
126,문화비축기지,37.571718,126.893246
64,올림픽공원,37.520934,127.122959



좌표 NaN 개수: 0 0
위도 범위: 37.4224347 ~ 37.699232
경도 범위: 126.8132285 ~ 127.1547791


In [100]:
# 날짜/시간 값 실제 형태 확인
date_col = "stdr_de_id"
hour_col = "tmzon_pd_se"

print("▶ pop rows:", len(pop))
print("고유 날짜 예시:", pop[date_col].dropna().astype(str).unique()[:5])
print("고유 시간 예시:", pop[hour_col].dropna().astype(str).unique()[:20])

print("\n▶ popxy 조인 전 pop rows:", len(pop))
print("▶ popxy_agg rows:", len(popxy_agg))

if len(popxy_agg) == 0:
    print("⚠️ popxy_agg 비어있음 → 코드/날짜/시간/인구컬럼 의심")

# 코드 길이 점검
print("\n코드 길이(pop code8):", pop["code8"].astype(str).str.len().value_counts().head())
if "code" in dongxy.columns:
    print("코드 길이(dongxy code):", dongxy["code"].astype(str).str.len().value_counts().head())
print("dongxy code8 예시:", dongxy["code8"].astype(str).head().tolist())

# 인구 분포
if "pop" in pop.columns:
    print("\npop 통계:\n", pop["pop"].describe())


▶ pop rows: 6784
고유 날짜 예시: ['20250810']
고유 시간 예시: ['07' '08' '09' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '20'
 '21' '22']

▶ popxy 조인 전 pop rows: 6784
▶ popxy_agg rows: 418

코드 길이(pop code8): code8
8    6784
Name: count, dtype: int64
코드 길이(dongxy code): code
12    891
Name: count, dtype: int64
dongxy code8 예시: ['11110515', '11110530', '11110540', '11110550', '11110560']

pop 통계:
 count      6784.000000
mean      46116.730091
std       23974.925708
min        5898.202400
25%       29677.620050
50%       40671.727400
75%       57452.161550
max      229444.934600
Name: pop, dtype: float64
